In [1]:
import os
import yaml
import json
import pandas as pd
from pathlib import Path

In [2]:
cwl_dir = '../cwl/'
Path(cwl_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
template = {
    'fq_1': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'fq_2': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'genome_lib_dir': {
        'class': 'Directory',
        'path': 'a/file/path'
    },
    'genome_db': {
        'class': 'Directory',
        'path': 'a/file/path'
    },
    'bwts': {
        'class': 'Directory',
        'path': 'a/file/path'
    },
    'integrate_executable': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'integrate_fasta': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'integrate_annotations': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'filter_database': {
        'class': 'Directory',
        'path': 'a/file/path'
    },
    'fusion_annotator_dir': {
        'class': 'Directory',
        'path': 'a/file/path'
    },
    'sample': 'a_string',
    'cpu': 'a_int'
}

In [3]:
import re

In [6]:
fp = 'path/to/a/file/sample.a.woo-boo.T.R1.fastq'
re.sub(r'^(.*).((fq)|(fastq))(.gz)?', r'\1', fp)

'path/to/a/file/sample.a.woo-boo.T.R1'

In [ ]:
cwl = {
    'class': 'CommandLineTool',
    'cwlVersion': 'v1.0',
    'id': 'adapter_trimming',
    'label': 'adapter_trimming',
    'baseCommand': ['python', '/pecgs-cnv/cnv/cnv.py'],
    'inputs': [
        {
            'id': 'sample',
            'type': 'string',
            'inputBinding': {
                'position': '1'
            }
        },
        {
            'id': 'tumor_bam',
            'type': 'File',
            'inputBinding': {
                'position': '2'
            }
        },
        {
            'id': 'normal_bam',
            'type': 'File',
            'inputBinding': {
                'position': '3'
            }
        },
        {
            'id': 'reference_dir',
            'type': 'Directory',

        },
        {
            'id': 'target_interval_list',
            'type': 'File',
            'inputBinding': {
                'prefix': '--target-interval-list',
                'position': '0'
            }
        },
        {
            'id': 'common_biallelic',
            'type': 'File',
            'inputBinding': {
                'prefix': '--common-biallelic',
                'position': '0'
            }
        },
        {
            'id': 'protein_coding_gene',
            'type': 'File',
            'inputBinding': {
                'prefix': '--protein-coding-gene',
                'position': '0'
            }
        },
        {
            'id': 'pool_of_normals',
            'type': 'File',
            'inputBinding': {
                'prefix': '--pool-of-normals',
                'position': '0'
            }
        },
        # needs path to be set so it works on compute1
        {
            'id': 'environ_PATH',
            'type': 'string?',
            'default': '/miniconda/envs/cnv/bin:$PATH'
        },
    ],
    'arguments': [
        {
            'position': 0,
            'prefix': '--genome',
            'valueFrom': '$(inputs.reference_dir)/GRCh38.d1.vd1.fa'
        },
        {
            'position': 0,
            'prefix': '--genome-dict',
            'valueFrom': '$(inputs.reference_dir)/GRCh38.d1.vd1.dict'
        },
        {
            'position': 0,
            'prefix': '--gene-level-script',
            'valueFrom': '/pecgs-cnv/cnv/segment_to_geneLevel.py'
        },
        {
            'position': 0,
            'prefix': '--merge-gene-script',
            'valueFrom': '/pecgs-cnv/cnv/mergeMultipleFilesToOne.py'
        },
        {
            'position': 0,
            'prefix': '--out-dir',
            'valueFrom': 'outputs'
        },
    ],
    'outputs': [
        {
            'id': 'gene_level_cnv',
            'type': 'File',
            'outputBinding': {'glob': 'outputs/*T.geneLevel.from_seg.cn'}
        }
    ],
    'requirements': [
        {
            'class': 'DockerRequirement',
            'dockerPull': 'estorrs/pecgs_cnv:0.0.1'
        },
        {
            'class': 'ResourceRequirement',
            'ramMin': 50000
        },
        {
            'class': 'EnvVarRequirement',
            'envDef': {
                'PATH': '$(inputs.environ_PATH)',
            }
        }
    ]
}

In [ ]:
t = {
    'class': 'Workflow',
    'cwlVersion': 'v1.0',
    'id': 'cnv_workflow',
    'label': 'cnv_workflow',
    'requirements': [{'class': 'ScatterFeatureRequirement'}],
    'inputs': [
        {
            'id': 'sample',
            'type': 'string',
        },
        {
            'id': 'tumor_bam',
            'type': 'File',
        },
        {
            'id': 'normal_bam',
            'type': 'File',
        },
        {
            'id': 'genome',
            'type': 'File',
        },
        {
            'id': 'genome_dict',
            'type': 'File',
        },
        {
            'id': 'target_interval_list',
            'type': 'File',
        },
        {
            'id': 'common_biallelic',
            'type': 'File',
        },
        {
            'id': 'protein_coding_gene',
            'type': 'File',
        },
        {
            'id': 'pool_of_normals',
            'type': 'File',
        },
    ],
    'outputs': [
        {
            'id': 'gene_level_cnv',
            'outputSource': 'cnv/gene_level_cnv',
            'type': 'File'
        },
    ],
    'steps': [
        {
            'id': 'cnv',
            'in': [
                {'id': 'genome', 'source': 'genome'},
                {'id': 'genome_dict', 'source': 'genome_dict'},
                {'id': 'target_interval_list', 'source': 'target_interval_list'},
                {'id': 'common_biallelic', 'source': 'common_biallelic'},
                {'id': 'protein_coding_gene', 'source': 'protein_coding_gene'},
                {'id': 'pool_of_normals', 'source': 'pool_of_normals'},
                {'id': 'sample', 'source': 'sample'},
                {'id': 'normal_bam', 'source': 'stage_normal_bam/output'},
                {'id': 'tumor_bam', 'source': 'stage_tumor_bam/output'}],
           'label': 'cnv',
           'out': [{'id': 'gene_level_cnv'}],
           'run': './cnv.cwl'
        },
        {
            'id': 'stage_normal_bam',
            'in': [{'id': 'BAM', 'source': 'normal_bam'}],
            'label': 'stage_normal_bam',
            'out': [{'id': 'output'}],
            'run': './stage_bam.cwl'
        },
        {
            'id': 'stage_tumor_bam',
            'in': [{'id': 'BAM', 'source': 'tumor_bam'}],
            'label': 'stage_tumor_bam',
            'out': [{'id': 'output'}],
            'run': './stage_bam.cwl'
        }
    ]
}